# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [50]:
# Dependencies and Setup
import pandas_bokeh
import hvplot.pandas
import geoviews as gv
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.tile_providers import Vendors
from bokeh.transform import factor_cmap
from bokeh.palettes import all_palettes
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [51]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,1.33,99,100,11.46,TF,1720808080
1,1,dolores,-36.3132,-57.6792,10.73,54,5,2.56,AR,1720808081
2,2,isetskoye,56.4858,65.3226,19.51,85,62,2.50,RU,1720808083
3,3,port elizabeth,-33.9180,25.5701,15.05,86,78,0.45,ZA,1720807834
4,4,blackmans bay,-43.0167,147.3167,6.40,87,75,0.45,AU,1720808085


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [52]:
output_notebook()

# Load your city data into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Convert City column to categorical
city_data_df['City'] = city_data_df['City'].astype('category')

# Create a ColumnDataSource
source = ColumnDataSource(city_data_df)

# Define a color mapper
unique_cities = city_data_df['City'].unique()
num_colors = len(unique_cities)
# Generate a larger custom palette
palette = all_palettes['Category20'][20] * (num_colors // 20 + 1)
palette = palette[:num_colors]

color_mapper = factor_cmap('City', palette=palette, factors=unique_cities.tolist())

# Create a figure
p = figure(width=1000, height=800, tools="pan,wheel_zoom,box_zoom,reset,hover,save",
           title="City Humidity Map", x_axis_label='Longitude', y_axis_label='Latitude')

# Add the map tiles  
p.add_tile(Vendors.CARTODBPOSITRON_RETINA)

# Add scatter glyphs
p.scatter('Lng', 'Lat', size=10, source=source, color=color_mapper, legend_field='City', fill_alpha=0.6)

# Configure the hover tool
hover = p.select_one(HoverTool)
hover.tooltips = [("City", "@City"), ("Country", "@Country"), ("Humidity", "@Humidity")]

# Adjust legend
p.legend.location = "right"
p.legend.click_policy = "hide"

# Show the plot
show(p)


Loading BokehJS ...

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[
    (city_data_df['Max Temp'] >= 20) & 
    (city_data_df['Max Temp'] <= 30) & 
    (city_data_df['Humidity'] < 50) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
204,204,lata,40.1629,-8.3327,24.95,49,0,3.74,PT,1720808334
311,311,chibougamau,49.9168,-74.3659,27.79,30,0,6.69,CA,1720808469
357,357,xai-xai,-25.0519,33.6442,20.91,34,0,2.87,MZ,1720808526
508,508,guijuelo,40.5576,-5.6707,29.05,19,0,4.13,ES,1720808705
524,524,veshenskaya,49.6303,41.7334,21.00,44,0,4.14,RU,1720808725


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
204,lata,PT,40.1629,-8.3327,49,
311,chibougamau,CA,49.9168,-74.3659,30,
357,xai-xai,MZ,-25.0519,33.6442,34,
508,guijuelo,ES,40.5576,-5.6707,19,
524,veshenskaya,RU,49.6303,41.7334,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, 'Hotel Name'] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
lata - nearest hotel: Residencial Botânico
chibougamau - nearest hotel: No hotel found
xai-xai - nearest hotel: No hotel found
guijuelo - nearest hotel: Hotel Torres
veshenskaya - nearest hotel: Дон


,City,Country,Lat,Lng,Humidity,Hotel Name
204,lata,PT,40.1629,-8.3327,49,Residencial Botânico
311,chibougamau,CA,49.9168,-74.3659,30,No hotel found
357,xai-xai,MZ,-25.0519,33.6442,34,No hotel found
508,guijuelo,ES,40.5576,-5.6707,19,Hotel Torres
524,veshenskaya,RU,49.6303,41.7334,44,Дон


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
# Enable the notebook output for bokeh
output_notebook()

# Configure the map plot
source = ColumnDataSource(hotel_df)

# Define a color mapper
unique_cities = hotel_df['City'].unique()
num_colors = len(unique_cities)

# Generate a larger custom palette
palette = all_palettes['Category20'][20] * (num_colors // 20 + 1)
palette = palette[:num_colors]

color_mapper = factor_cmap('City', palette=palette, factors=unique_cities.tolist())

# Create a figure
p = figure(width=1000, height=800, tools="pan,wheel_zoom,box_zoom,reset,hover,save",
           title="City Humidity Map with Hotels", x_axis_label='Longitude', y_axis_label='Latitude')

# Add the map tiles
p.add_tile(Vendors.CARTODBPOSITRON_RETINA)

# Add scatter glyphs
p.scatter('Lng', 'Lat', size=10, source=source, color=color_mapper, legend_field='City', fill_alpha=0.6)

# Configure the hover tool
hover = p.select_one(HoverTool)
hover.tooltips = [("City", "@City"), ("Country", "@Country"), ("Humidity", "@Humidity"), ("Hotel", "@{Hotel Name}")]

# Adjust legend
p.legend.location = "right"
p.legend.click_policy = "hide"

# Show the plot
show(p)



Loading BokehJS ...